In [69]:
import pandas as pd
import numpy as np
import ast

In [91]:
movies  = pd.read_csv('imdb_top_1000.csv',skip_blank_lines='True')


In [103]:
movies 

,Series_Title,tags
0,The Shawshank Redemption,"[Drama, Twoimprisonedmenbondoveranumberofyears..."
1,The Godfather,"[Crime, Drama, Anorganizedcrimedynasty'sagingp..."
2,The Dark Knight,"[Action, Crime, Drama, Whenthemenaceknownasthe..."
3,The Godfather: Part II,"[Crime, Drama, TheearlylifeandcareerofVitoCorl..."
4,12 Angry Men,"[Crime, Drama, Ajuryholdoutattemptstopreventam..."
...,...,...
995,Breakfast at Tiffany's,"[Comedy, Drama, Romance, AyoungNewYorksocialit..."
996,Giant,"[Drama, Western, Sprawlingepiccoveringthelifeo..."
997,From Here to Eternity,"[Drama, Romance, War, InHawaiiin1941,aprivatei..."
998,Lifeboat,"[Drama, War, Severalsurvivorsofatorpedoedmerch..."


In [93]:
movies['Cast'] = movies[['Star1', 'Star2', 'Star3', 'Star4']].values.tolist()

In [94]:
movies = movies.drop('Star1', axis = True)
movies = movies.drop('Star2', axis = True)
movies = movies.drop('Star3', axis = True)
movies = movies.drop('Star4', axis = True)


In [95]:
movies = movies[['Series_Title','Overview','Genre','Cast','Director','IMDB_Rating']]

In [96]:
movies['Overview'] = movies['Overview'].apply(lambda x: x.replace(" ",""))
movies['Director'] = movies['Director'].apply(lambda x: x.replace(" ",""))

In [97]:
movies['Genre'] = movies['Genre'].str.split(', ')
movies['Overview'] = movies['Overview'].str.split(', ')  
movies['Director'] = movies['Director'].str.split(', ')  


In [99]:
movies['tags'] =  movies['Genre'] + movies['Overview'] + movies['Cast'] + movies['Director']

In [102]:
movies = movies[['Series_Title','tags']]

In [104]:
movies['tags'] = movies['tags'].apply(lambda x: [item.lower() for item in x])

/tmp/ipykernel_3425/45140674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x: [item.lower() for item in x])


In [105]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x)) 

/tmp/ipykernel_3425/2127225556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))


In [106]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [107]:
vector = cv.fit_transform(movies['tags']).toarray()

In [108]:
vector.shape

(1000, 5000)

In [109]:
from sklearn.metrics.pairwise import cosine_similarity

In [110]:
similarity = cosine_similarity(vector)

In [111]:
similarity

array([[1.        , 0.09128709, 0.0877058 , ..., 0.08164966, 0.19069252,
        0.        ],
       [0.09128709, 1.        , 0.16012815, ..., 0.0745356 , 0.08703883,
        0.0745356 ],
       [0.0877058 , 0.16012815, 1.        , ..., 0.07161149, 0.0836242 ,
        0.07161149],
       ...,
       [0.08164966, 0.0745356 , 0.07161149, ..., 1.        , 0.15569979,
        0.        ],
       [0.19069252, 0.08703883, 0.0836242 , ..., 0.15569979, 1.        ,
        0.07784989],
       [0.        , 0.0745356 , 0.07161149, ..., 0.        , 0.07784989,
        1.        ]], shape=(1000, 1000))

In [114]:
def recommend(movie):
    index = movies[movies['Series_Title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(movies.iloc[i[0]].Series_Title)
        

In [115]:
recommend('Batman Begins')

The Dark Knight
The Prestige
The Dark Knight Rises
Inception
Letters from Iwo Jima


In [116]:
recommend('Avatar')

Aliens
Ghostbusters
Guardians of the Galaxy
Wo hu cang long
Guardians of the Galaxy Vol. 2


In [118]:
recommend('Inception')

Batman Begins
The Revenant
Letters from Iwo Jima
Serenity
Edge of Tomorrow


In [119]:


import pickle
pickle.dump(movies,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))